In [1]:
#Eddie Costantini~Pitch Prediction Problem for Swish Analytics
#import packages
import numpy as np
import pandas as pd

In [2]:
from pybaseball import statcast
#data = statcast(start_dt='2018-03-29', end_dt='2018-08-31')
data = statcast(start_dt='2018-04-01', end_dt='2018-04-15')
data.head()

This is a large query, it may take a moment to complete
Completed sub-query from 2018-04-01 to 2018-04-06
Completed sub-query from 2018-04-07 to 2018-04-12
Completed sub-query from 2018-04-13 to 2018-04-15


,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,157,SI,2018-04-15,94.1,4.4012,5.1380,Jake Diekman,545350.0,518617.0,strikeout,...,1.0,3.0,1.0,3.0,3.0,1.0,1.0,3.0,Standard,Standard
1,166,SI,2018-04-15,94.3,4.2906,5.2711,Jake Diekman,545350.0,518617.0,NaN,...,1.0,3.0,1.0,3.0,3.0,1.0,1.0,3.0,Standard,Standard
2,171,SI,2018-04-15,94.0,4.3510,5.2729,Jake Diekman,545350.0,518617.0,NaN,...,1.0,3.0,1.0,3.0,3.0,1.0,1.0,3.0,Standard,Standard
3,180,SI,2018-04-15,93.0,4.3187,5.3569,Jake Diekman,545350.0,518617.0,NaN,...,1.0,3.0,1.0,3.0,3.0,1.0,1.0,3.0,Standard,Standard
4,185,SI,2018-04-15,93.1,4.3064,5.1386,Jake Diekman,545350.0,518617.0,NaN,...,1.0,3.0,1.0,3.0,3.0,1.0,1.0,3.0,Standard,Standard


In [3]:
data.to_csv("StatcastTest.csv")

In [6]:
# #import data set
# fileIn = 'Statcast2018'
# dfIn = pd.read_csv(fileIn)

# dfIn.head()

In [4]:
dfIn = data[['pitch_type','player_name','batter','pitcher',
           'stand','p_throws','balls','strikes','outs_when_up',
            'inning','at_bat_number','pitch_number','bat_score','fld_score',
          'on_1b','on_2b','on_3b','if_fielding_alignment','of_fielding_alignment',
          'plate_x','plate_z','zone','release_speed','type',
          'estimated_woba_using_speedangle']]

#flip it

df=dfIn.iloc[::-1]

df.head()

,pitch_type,player_name,batter,pitcher,stand,p_throws,balls,strikes,outs_when_up,inning,...,on_2b,on_3b,if_fielding_alignment,of_fielding_alignment,plate_x,plate_z,zone,release_speed,type,estimated_woba_using_speedangle
55550,FT,Ryan Carpenter,543281.0,543001.0,R,L,0.0,0.0,0.0,1.0,...,NaN,NaN,Standard,Standard,0.9165,2.8582,12.0,90.4,B,NaN
55549,FF,Ryan Carpenter,543281.0,543001.0,R,L,1.0,0.0,0.0,1.0,...,NaN,NaN,Standard,Standard,-0.5425,3.4360,1.0,89.5,S,NaN
55548,CH,Ryan Carpenter,543281.0,543001.0,R,L,1.0,1.0,0.0,1.0,...,NaN,NaN,Standard,Standard,0.6890,1.8333,9.0,82.9,X,0.769
55547,FT,Ryan Carpenter,474568.0,543001.0,R,L,0.0,0.0,0.0,1.0,...,NaN,NaN,Standard,Standard,0.5321,2.9204,6.0,90.1,S,NaN
55546,FT,Ryan Carpenter,474568.0,543001.0,R,L,0.0,1.0,0.0,1.0,...,NaN,NaN,Standard,Standard,0.4832,2.3416,6.0,91.2,S,NaN


In [5]:
#columns to clean
#combine stand and p_throws into same_hand - if same then 1, otherwise 0
df["same_hand"] = np.where(df['stand']==df['p_throws'], 1, 0)
df=df.drop(columns=['stand', 'p_throws'])

#combine away_team_runs, home_team_runs, and top into score_diff
df['score_diff'] = df['fld_score']-df['bat_score']
df=df.drop(columns=['fld_score', 'bat_score'])

#change on_1b, on_2b, on_3b to binary value
df['on_1'] = df['on_1b'].notnull().astype(int)
df['on_2'] = df['on_2b'].notnull().astype(int)
df['on_3'] = df['on_3b'].notnull().astype(int)
df=df.drop(columns=['on_1b', 'on_2b','on_3b'])

df.head()

C:\Users\ecost\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,pitch_type,player_name,batter,pitcher,balls,strikes,outs_when_up,inning,at_bat_number,pitch_number,...,plate_z,zone,release_speed,type,estimated_woba_using_speedangle,same_hand,score_diff,on_1,on_2,on_3
55550,FT,Ryan Carpenter,543281.0,543001.0,0.0,0.0,0.0,1.0,1.0,1.0,...,2.8582,12.0,90.4,B,NaN,0,0.0,0,0,0
55549,FF,Ryan Carpenter,543281.0,543001.0,1.0,0.0,0.0,1.0,1.0,2.0,...,3.4360,1.0,89.5,S,NaN,0,0.0,0,0,0
55548,CH,Ryan Carpenter,543281.0,543001.0,1.0,1.0,0.0,1.0,1.0,3.0,...,1.8333,9.0,82.9,X,0.769,0,0.0,0,0,0
55547,FT,Ryan Carpenter,474568.0,543001.0,0.0,0.0,0.0,1.0,2.0,1.0,...,2.9204,6.0,90.1,S,NaN,0,0.0,1,0,0
55546,FT,Ryan Carpenter,474568.0,543001.0,0.0,1.0,0.0,1.0,2.0,2.0,...,2.3416,6.0,91.2,S,NaN,0,0.0,1,0,0


In [6]:
#LAG 1
#Use NaN to show first pitch of each at-bat does not have a preceding pitch type or zone
df.loc[df['pitch_number'] == 1, 'lag1_type'] = "Na"
df.loc[df['pitch_number'] == 1, 'lag1_zone'] = "Na"
df.loc[df['pitch_number'] == 1, 'lag1_outcome'] = "Na"

#Previous Pitch in at-bat
df.loc[df['pitch_number'] >= 2, 'lag1_type'] = df['pitch_type'].shift(1)
df.loc[df['pitch_number'] >= 2, 'lag1_x'] = df['plate_x'].shift(1)
df.loc[df['pitch_number'] >= 2, 'lag1_z'] = df['plate_z'].shift(1)
df.loc[df['pitch_number'] >= 2, 'lag1_zone'] = df['zone'].shift(1)
df.loc[df['pitch_number'] >= 2, 'lag1_outcome'] = df['type'].shift(1)

#LAG 2
df.loc[df['pitch_number'] <= 2, 'lag2_type'] = "Na"
df.loc[df['pitch_number'] <= 2, 'lag2_zone'] = "Na"
df.loc[df['pitch_number'] <= 2, 'lag2_outcome'] = "Na"

df.loc[df['pitch_number'] >= 3, 'lag2_type'] = df['pitch_type'].shift(2)
df.loc[df['pitch_number'] >= 3, 'lag2_x'] = df['plate_x'].shift(2)
df.loc[df['pitch_number'] >= 3, 'lag2_z'] = df['plate_z'].shift(2)
df.loc[df['pitch_number'] >= 3, 'lag2_zone'] = df['zone'].shift(2)
df.loc[df['pitch_number'] >= 3, 'lag2_outcome'] = df['type'].shift(2)

#LAG 3
df.loc[df['pitch_number'] <= 3, 'lag3_type'] = "Na"
df.loc[df['pitch_number'] <= 3, 'lag3_zone'] = "Na"
df.loc[df['pitch_number'] <= 3, 'lag3_outcome'] = "Na"

df.loc[df['pitch_number'] >= 4, 'lag3_type'] = df['pitch_type'].shift(3)
df.loc[df['pitch_number'] >= 4, 'lag3_x'] = df['plate_x'].shift(3)
df.loc[df['pitch_number'] >= 4, 'lag3_z'] = df['plate_z'].shift(3)
df.loc[df['pitch_number'] >= 4, 'lag3_zone'] = df['zone'].shift(3)
df.loc[df['pitch_number'] >= 4, 'lag3_outcome'] = df['type'].shift(3)


In [7]:
#drop current pitch x,z,zone columns, as they are not available at time of the pitch
df=df.drop(columns=['plate_x','plate_z','zone'])

In [19]:
#SP/RP feature creation, shift data

In [8]:
#export analysis results to CSV file
df.to_csv('clean_pitch_data_test.csv')